<a href="https://colab.research.google.com/github/yp6969/Inro_to_DL/blob/main/Text_classifiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U pip
! pip install -U torch==1.5.0
! pip install -U torchtext==0.6.0
! pip install -U matplotlib==3.2.1
! pip install -U clearml>=0.15.0
! pip install -U tensorboard==2.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.0/752.0 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.5.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.0 which is incom

In [ ]:
import os
import time
import re
import torch
import torch.nn as nn
from torchtext.datasets import text_classification
from torch.utils.tensorboard import SummaryWriter
import torchtext
from clearml import Task

%matplotlib inline

In [ ]:
NGRAMS = 2
if not os.path.isdir('./.data'):
	os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ag_news_csv.tar.gz: 100%|██████████| 11.8M/11.8M [00:00<00:00, 29.6MB/s]
120000lines [00:09, 12234.59lines/s]
120000lines [00:18, 6393.76lines/s]
7600lines [00:01, 6873.13lines/s]


In [ ]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextSentiment, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)

device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
print('Device to use: {}'.format(device))
model.to(device)

Device to use: cpu


TextSentiment(
  (embedding): EmbeddingBag(1308844, 32, mode=mean)
  (fc): Linear(in_features=32, out_features=4, bias=True)
)

In [ ]:
tensorboard_writer = SummaryWriter('./tensorboard_logs')


In [ ]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [ ]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()
    
    return train_loss / len(sub_train_), train_acc / len(sub_train_)


In [ ]:
def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

## training

In [ ]:
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 27 seconds
	Loss: 0.0261(train)	|	Acc: 84.7%(train)
	Loss: 0.0000(valid)	|	Acc: 90.0%(valid)
Epoch: 2  | time in 0 minutes, 27 seconds
	Loss: 0.0120(train)	|	Acc: 93.6%(train)
	Loss: 0.0000(valid)	|	Acc: 91.3%(valid)
Epoch: 3  | time in 0 minutes, 26 seconds
	Loss: 0.0070(train)	|	Acc: 96.3%(train)
	Loss: 0.0000(valid)	|	Acc: 91.6%(valid)
Epoch: 4  | time in 0 minutes, 26 seconds
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0000(valid)	|	Acc: 91.9%(valid)
Epoch: 5  | time in 0 minutes, 29 seconds
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0000(valid)	|	Acc: 91.9%(valid)


In [ ]:
def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

In [28]:
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}


str1 = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."



str2 = "Pilgrims have been told to use a website to register their interest and that a draw will take place to choose who will go on Hajj.\
Some have already booked with Hajj tour operators and could lose their money.\
Hajj is a religious requirement for all Muslims who can afford it.\
It is one of the five pillars of Islam along with five daily prayers, fasting, charity and a belief in God (Allah) and in Muhammad as a prophet.\
The cost of the once in a lifetime trip varies from about £6,000 to more than £10,000 and many Muslims spend years saving up for it."


str3 = "Quantum computers can make very complex calculations extremely quickly and their creators say they can solve the problems regular computers cannot.\
        The MoD will work with British company Orca Computing to explore applications for quantum technology in defence."

str4 = "Watching Scotland getting eaten whole by a hungry Ireland team at.\
 the Aviva on Saturday summoned up an image from one of those David Attenborough.\
  programmes, where the cocksure wildebeest wanders into the wrong part of the.\
   Serengeti only to be pounced on by a ravenous lion."



In [29]:
vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(str1, model, vocab, 2)])
print("This is a %s news" %ag_news_label[predict(str2, model, vocab, 2)])
print("This is a %s news" %ag_news_label[predict(str3, model, vocab, 2)])
print("This is a %s news" %ag_news_label[predict(str4, model, vocab, 2)])

This is a Sports news
This is a World news
This is a Sci/Tec news
This is a Sports news
